# **Importing Packages**

In [ ]:
import numpy as np
import pandas as pd
import nltk
import tensorflow as tf
import keras


# **Input dataset**

In [ ]:
df = pd.read_csv("/content/dataset.csv")
df.head()

UnicodeDecodeError: ignored

In [ ]:
df.info()

In [ ]:
#Converting all text to lowercase
df["Text"] = df["Text"].str.lower()
df.head()

In [ ]:
max_words = 50000
max_len = 100

# **Tokenization**

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(list(df['Text']))
train_df = tokenizer.texts_to_sequences(list(df['Text']))

In [ ]:
train_df = tf.keras.preprocessing.sequence.pad_sequences(train_df,maxlen = max_len)

In [ ]:
len(tokenizer.word_index)

In [ ]:
Y = df['language']
Y

# **Encoding**

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
le = preprocessing.LabelEncoder()

In [ ]:
le.fit(Y)

In [ ]:
list(le.classes_)

In [ ]:
Y2 = le.fit_transform(Y)
total_languages = df['language'].nunique()
Y2 = keras.utils.to_categorical(Y2,num_classes=total_languages)
np.shape(Y2)

# **Training the model**

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(train_df,Y2)
embedding_dims = 500
vocab_size = len(tokenizer.word_index)+1
model = tf.keras.Sequential([tf.keras.layers.Embedding(input_dim = vocab_size,output_dim = embedding_dims,input_length = max_len),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(total_languages,activation=tf.nn.softmax)
                            ])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer ='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(np.array(X_train),np.array(Y_train),epochs=3)

In [ ]:
model.evaluate(np.array(X_test),np.array(Y_test))






#Positive Test Cases

#MultiLang : Swedish and Dutch
#text = ["När du väl känner till alla element är det inte svårt att ta ihop en mening. Als je eenmaal alle elementen kent, is het niet moeilijk om een zin samen te stellen. "]


#Multilang : Hindi and Arabic
text = [" بمجرد أن تعرف كل العناصر ، ليس من الصعب تجميع جملة معًاएक बार जब आप सभी तत्वों को जान लेते हैं, तो एक वाक्य को एक साथ खींचना मुश्किल नहीं है।"]

#Negative Test Case
text=["Une incorporation de mots est une classe d'approches permettant de représenter des mots et des documents à l'aide d'une représentation vectorielle dense.Il s'agit d'une amélioration par rapport aux schémas de codage de modèle de sac de mots traditionnels où de grands vecteurs épars étaient utilisés pour représenter chaque mot ou pour marquer chaque mot dans un vecteur pour représenter un vocabulaire entier. Ces représentations étaient rares car les vocabulaires étaient vastes et un mot ou un document donné serait représenté par un grand vecteur composé principalement de valeurs nulles.Au lieu de cela, dans une incorporation, les mots sont représentés par des vecteurs denses où un vecteur représente la projection du mot dans un espace vectoriel continu.La position d'un mot dans l'espace vectoriel est apprise à partir du texte et est basée sur les mots qui entourent le mot lorsqu'il est utilisé.La position d'un mot dans l'espace vectoriel appris est appelée son incorporation.Voici deux exemples populaires de méthodes d'apprentissage de l'intégration deएक बार जब आप सभी तत्वों को जान एक शब्द एम्बेडिंग घने वेक्टर प्रतिनिधित्व का उपयोग करते हुए शब्दों और दस्तावेजों का प्रतिनिधित्व करने के लिए दृष्टिकोणों का एक वर्ग है।यह अधिक पारंपरिक बैग-ऑफ-वर्ड मॉडल एन्कोडिंग योजनाओं पर एक सुधार है जहां बड़े विरल वैक्टर का उपयोग प्रत्येक शब्द का प्रतिनिधित्व करने के लिए या वेक्टर के भीतर प्रत्येक शब्द को पूरी शब्दावली का प्रतिनिधित्व करने के लिए किया जाता था। ये प्रतिनिधित्व विरल थे क्योंकि शब्दसंग्रह विशाल थे और किसी दिए गए शब्द या दस्तावेज को एक बड़े वेक्टर द्वारा दर्शाया जाएगा जिसमें ज्यादातर शून्य मान शामिल थे।इसके बजाय, एक एम्बेडिंग में, शब्दों को घने वैक्टर द्वारा दर्शाया जाता है जहां एक वेक्टर एक निरंतर वेक्टर अंतरिक्ष में शब्द के प्रक्षेपण का प्रतिनिधित्व करता है।"]


## **TEST CASES**
# Inline single language 


In [ ]:
#ENGLISH
text = [" creation of a word essential for encoding A byproduct of words  in our Une incorporation de mots est une classe d'approches permettant de représenter"]


In [ ]:
test_text = tokenizer.texts_to_sequences(text)
test_text = tf.keras.preprocessing.sequence.pad_sequences(test_text, maxlen=max_len)
predictions = model.predict(test_text)
out = predictions.argmax()
print(le.inverse_transform([out]))
print(predictions)


# **Dual Language Identification**

In [ ]:

#Multilang : French
text = [" Une incorporation de mots est une classe d'approches permettant de représenter"]

test_text = tokenizer.texts_to_sequences(text)
test_text = tf.keras.preprocessing.sequence.pad_sequences(test_text, maxlen=max_len)


In [ ]:
#------------------Identify two languages by finding the two highest values---------------------#
#firstlang
predictions = model.predict(test_text)
out = predictions.argmax()
print(out)
print(le.inverse_transform([out]))
#converting it to a list
a = predictions[0]
arr = list()
for i in a:
  arr.append(i)
#secondlang
arr[out]=0
out2=arr.index(max(arr))
print(out2)
print(le.inverse_transform([out2]))

# **Multi-Language Detection**

In [ ]:
!pip install langdetect
from langdetect import detect


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
all_languages_codes =   {
  
  	'ar': 'Arabic',
  	'ara': 'Arabic',
  	'zh': 'Chinese',
  	'zh-cn': 'Chinese (Simplified)',
  	'zh-tw': 'Chinese (Traditional)',
  	'chi/zho*': 'Chinese',
  	'nl': 'Dutch',
  	'dut/nld*': 'Dutch',
  	'en': 'English',
  	'eng': 'English',
    'et': 'Estonian',
  	'est': 'Estonian',
  	'fr': 'French',
  	'fre/fra*': 'French',
   	'de': 'German',
  	'ger/deu*': 'German',
  	'hi': 'Hindi',
  	'hin': 'Hindi',
  	'ja': 'Japanese',
  	'jpn': 'Japanese',
  	'ko': 'Korean',
  	'kor': 'Korean',
  	'la': 'Latin',
  	'lat': 'Latin',
  	'fa': 'Persian',
  	'per/fas*': 'Persian',
  	'pt': 'Portuguese',
  	'por': 'Portuguese',
  	'ps': 'Pushto',
  	'pus': 'Pushto',
  	'ro': 'Romanian',
  	'rum/ron*': 'Romanian',
  	'ru': 'Russian',
  	'rus': 'Russian',
  	'es': 'Spanish; Castilian',
  	'spa': 'Spanish; Castilian',
   	'sv': 'Swedish',
  	'swe': 'Swedish',
  	'ta': 'Tamil',
  	'tam': 'Tamil',
  	'th': 'Thai',
  	'tha': 'Thai',
  	'tr': 'Turkish',
  	'tur': 'Turkish',
   	'ur': 'Urdu',
  	'urd': 'Urdu',
  	
  }

In [ ]:
testsite_array = []
with open('/input .txt') as my_file:
    for line in my_file:
        testsite_array.append(line)


In [ ]:
Column_names = ["Language Code", "Input Sentence"]
format_text = '{:24}' * (len(Column_names) + 1)

In [ ]:
print ('\n', format_text.format("Language Name", *Column_names),'\n','='*120)
for data in testsite_array:
    language_code = detect(data)    
    sentence = [all_languages_codes.get(language_code), language_code, data]
    print (format_text.format(*sentence))




    